<a href="https://colab.research.google.com/github/newmantic/barrier_option/blob/main/barrier_option.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class BarrierOption:
    def __init__(self, option_type, barrier_type, initial_price, strike_price, barrier_level, maturity, risk_free_rate, volatility, rebate=0):
        self.option_type = option_type  # 'call' or 'put'
        self.barrier_type = barrier_type  # 'knock_in' or 'knock_out'
        self.initial_price = initial_price
        self.strike_price = strike_price
        self.barrier_level = barrier_level
        self.maturity = maturity
        self.risk_free_rate = risk_free_rate
        self.volatility = volatility
        self.rebate = rebate  # Rebate paid if the option is knocked out (optional)

    def simulate_price_path(self, n_steps, n_scenarios):
        dt = self.maturity / n_steps
        price_paths = np.zeros((n_steps + 1, n_scenarios))
        price_paths[0] = self.initial_price

        for t in range(1, n_steps + 1):
            z = np.random.standard_normal(n_scenarios)
            price_paths[t] = price_paths[t - 1] * np.exp((self.risk_free_rate - 0.5 * self.volatility ** 2) * dt + self.volatility * np.sqrt(dt) * z)

        return price_paths

    def is_knocked_in(self, price_path):
        return np.any(price_path >= self.barrier_level) if self.option_type == 'call' else np.any(price_path <= self.barrier_level)

    def is_knocked_out(self, price_path):
        return np.any(price_path >= self.barrier_level) if self.option_type == 'call' else np.any(price_path <= self.barrier_level)

    def calculate_payoff(self, price_path):
        final_price = price_path[-1]

        if self.barrier_type == 'knock_in':
            if self.is_knocked_in(price_path):
                return max(final_price - self.strike_price, 0) if self.option_type == 'call' else max(self.strike_price - final_price, 0)
            else:
                return self.rebate
        elif self.barrier_type == 'knock_out':
            if self.is_knocked_out(price_path):
                return self.rebate
            else:
                return max(final_price - self.strike_price, 0) if self.option_type == 'call' else max(self.strike_price - final_price, 0)

    def run_simulation(self, n_steps=252, n_scenarios=10000):
        price_paths = self.simulate_price_path(n_steps, n_scenarios)
        payoffs = np.zeros(n_scenarios)

        for i in range(n_scenarios):
            payoffs[i] = self.calculate_payoff(price_paths[:, i])

        return np.mean(payoffs) * np.exp(-self.risk_free_rate * self.maturity)

# Example usage
if __name__ == "__main__":
    option = BarrierOption(
        option_type='call',
        barrier_type='knock_in',
        initial_price=100,
        strike_price=105,
        barrier_level=110,
        maturity=1,
        risk_free_rate=0.02,
        volatility=0.2
    )

    expected_payoff = option.run_simulation()
    print(f"Expected Payoff: {expected_payoff:.2f}")

Expected Payoff: 6.70
